### Analisis de grupo de Whatsapp

In [9]:
# Cargar Librerias

import pandas as pd
import re
import numpy as np
import collections
import emoji
import plotly.express as px
import matplotlib.pyplot as plt

In [2]:
#Funciones de recuperación y limpieza

def starts_with_date_time(s):
    # regex pattern for date
    pattern = r'^\d{1,2}/\d{1,2}/\d{4}\s\d{1,2}:\d{2}\s[ap].\sm\.'
    result = re.match(pattern, s)
    if result:
        return True
    return False

In [3]:
# Finds username of any given format.
def find_author(s):
    patterns = [
        '([\w]+):',                        # First Name
        '([\w]+[\s]+[\w]+):',              # First Name + Last Name
        '([\w]+[\s]+[\w]+[\s]+[\w]+):',    # First Name + Middle Name + Last Name
        '([+]\d{2} \d{3} \d{3} \d{4}):',   # Mobile Number (US)
        '[\w]+ ?[^\s\u1f300-\u1f5ff]:',    # Name and Emoji              
    ]
    pattern = '^' + '|'.join(patterns)
    result = re.match(pattern, s)
    if result:
        return True
    return False

In [4]:
def get_data_point(line):   
    split_line = line.split(' - ') 
    date_time = split_line[0].split(' ') 
    date = date_time[0]
    time = ' '.join(date_time[1:])
    message = ' '.join(split_line[1:])
    if find_author(message): 
        split_message = message.split(': ') 
        author = split_message[0] 
        message = ' '.join(split_message[1:])
    else:
        author = None
    return date, time, author, message

In [5]:
#Genera una lista con los datos para convertilos en data frame

parsed_data = [] 
conversation_path = "Chat.txt" # chat file
with open(conversation_path, encoding='utf8') as fp:
    fp.readline() # Skipping first line of the file 
    message_buffer = [] 
    date, time, author = None, None, None
    while True:
        line = fp.readline() 
        if not line: 
            break
        line = line.strip() 
        if starts_with_date_time(line): 
            print(line)
            if len(message_buffer) > 0: 
                parsed_data.append([date, time, author, ' '.join(message_buffer)]) 
            message_buffer.clear() 
            date, time, author, message = get_data_point(line) 
            message_buffer.append(message) 
        else:
            message_buffer.append(line)


4/9/2018 2:30 p. m. - ‎Ines Trabajo creó el grupo "Las mas Bellas de Dirego"
28/6/2019 2:19 p. m. - ‎Ines Dirego te añadió
28/6/2019 2:19 p. m. - Ines Dirego: conta
28/6/2019 2:19 p. m. - Ines Dirego: ya pasates la prueba
28/6/2019 2:19 p. m. - Cony Dirego: de fresa ines??
28/6/2019 2:20 p. m. - Edna Dirego: Jajajajajaj
28/6/2019 2:20 p. m. - Ines Dirego: cafres
28/6/2019 2:20 p. m. - Ines Dirego: Carrizo ese q tiene nieve  y todo el pedo
28/6/2019 2:20 p. m. - Cony Dirego: ines dime
28/6/2019 2:20 p. m. - Cony Dirego: va pedir
28/6/2019 2:21 p. m. - Ines Dirego: cafrizz de ese q tiene nieve
28/6/2019 2:21 p. m. - Cony Dirego: ya entrego el cel
28/6/2019 2:21 p. m. - Cony Dirego: con chokolate o sin chokolate?
28/6/2019 2:21 p. m. - Ines Dirego: con
28/6/2019 2:22 p. m. - Cony Dirego: Edna tu no??
28/6/2019 2:22 p. m. - Edna Dirego: No yo no
28/6/2019 2:22 p. m. - Edna Dirego: Me voy a quedar loca. Ya me tome uno
28/6/2019 2:22 p. m. - Ines Dirego: ella compro en la mañana con su amigu

4/1/2020 2:30 p. m. - Ines Dirego: 🤗🤗
4/1/2020 9:54 p. m. - Cony Dirego: <Multimedia omitido>
4/1/2020 9:54 p. m. - Cony Dirego: X favor panaderia
4/1/2020 9:54 p. m. - Cony Dirego: Jajaja
6/1/2020 7:25 p. m. - Cony Dirego: Ssoy una gordaaa
6/1/2020 7:25 p. m. - Cony Dirego: <Multimedia omitido>
6/1/2020 8:34 p. m. - Araceli G.R.: 😂😂😂😂
9/1/2020 1:43 p. m. - Cony Dirego: <Multimedia omitido>
9/1/2020 1:44 p. m. - Araceli G.R.: 😂😂😂😂
10/1/2020 7:21 p. m. - Cony Dirego: <Multimedia omitido>
10/1/2020 7:21 p. m. - Cony Dirego: Q bueno q estudié
10/1/2020 7:21 p. m. - Cony Dirego: 😂😂
10/1/2020 7:21 p. m. - Adriana Rocha Dirego: Jajajajajajaja siii
10/1/2020 7:21 p. m. - Ines Dirego: jajajaj
10/1/2020 7:22 p. m. - Ines Dirego: Ramón me deposito 200
10/1/2020 7:22 p. m. - Ines Dirego: hoy
10/1/2020 7:22 p. m. - Ines Dirego: ya van os semana sseguidas
10/1/2020 7:22 p. m. - Cony Dirego: Amonoos
10/1/2020 7:22 p. m. - Cony Dirego: Jajajajja
10/1/2020 7:23 p. m. - Ines Dirego: jajaja
10/1/2020 7:

3/8/2020 8:29 p. m. - +52 662 290 4710: aaaah no
3/8/2020 8:29 p. m. - Ines Dirego: Jajaja
3/8/2020 8:29 p. m. - Ines Dirego: Chupacela nomas
3/8/2020 8:29 p. m. - +52 662 290 4710: Hasta qué se case conmigo
3/8/2020 8:29 p. m. - Adriana Rocha Dirego: Supcionalo
3/8/2020 8:29 p. m. - Ines Dirego: Cayateeee
3/8/2020 8:29 p. m. - +52 662 290 4710: Jajajajaja jajajaa
3/8/2020 8:30 p. m. - +52 662 290 4710: Jajajajajajajaja
3/8/2020 8:30 p. m. - Ines Dirego: Mejor primero chupacela no valla salir con una pequeñes
3/8/2020 8:30 p. m. - Adriana Rocha Dirego: Chetincito
3/8/2020 8:30 p. m. - Ines Dirego: Jajaja
3/8/2020 8:30 p. m. - Adriana Rocha Dirego: Vale x un chetin
3/8/2020 8:30 p. m. - Adriana Rocha Dirego: Te acuerdas
3/8/2020 8:30 p. m. - Adriana Rocha Dirego: En tu cumple JAJA
3/8/2020 8:30 p. m. - Ines Dirego: Jajaja
3/8/2020 8:30 p. m. - Ines Dirego: Si
3/8/2020 8:31 p. m. - +52 662 290 4710: Jajajajajjaja
3/8/2020 8:33 p. m. - Adriana Rocha Dirego: <Multimedia omitido>
3/8/2020 8

In [6]:
# Genera el dataframe

df = pd.DataFrame(parsed_data, columns=['Date', 'Time', 'Author', 'Message'])
df["Date"] = pd.to_datetime(df["Date"])

In [38]:
df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2257 entries, 0 to 2256
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Date     2257 non-null   datetime64[ns]
 1   Time     2257 non-null   object        
 2   Author   2128 non-null   object        
 3   Message  2257 non-null   object        
dtypes: datetime64[ns](1), object(3)
memory usage: 70.7+ KB


,Date,Time,Author,Message
0,2018-04-09,2:30 p. m.,None,"‎Ines Trabajo creó el grupo ""Las mas Bellas de..."
1,2019-06-28,2:19 p. m.,None,‎Ines Dirego te añadió
2,2019-06-28,2:19 p. m.,Ines Dirego,conta
3,2019-06-28,2:19 p. m.,Ines Dirego,ya pasates la prueba
4,2019-06-28,2:19 p. m.,Cony Dirego,de fresa ines??
...,...,...,...,...
2252,2020-10-10,8:53 p. m.,Cony Dirego,Ohhh
2253,2020-10-10,8:53 p. m.,Cony Dirego,Tbn ombre
2254,2020-10-10,8:53 p. m.,Cony Dirego,Una ves al año
2255,2020-10-10,8:53 p. m.,Ines Dirego,Yace


In [7]:
df["Date"] = pd.to_datetime(df["Date"], format= "%d/%m/%y") # Formato de fecha
df['Message'] = df['Message'].str.lower() # Mensajes en minúsculas
df['Message'] = df.Message.str.replace(r"(a|j)?(ja)+(a|j)?", "jaja") # Estandarizamos la expresión de risa a "jaja"
df = df.dropna() # Para eliminar entradas en blanco (p.ej. los mensajes de sistema no tienen autor)

In [8]:
# Lista con los personajes del Dc
got_dt = pd.read_csv("DCpersonajes.csv") 

# Obtiene nombres y aliases
nombres = list(df.Author.unique())
aliases = list(got_dt.name.sample(len(nombres)))

df.Author.replace(nombres, aliases, inplace=True) # Remplaza en la reemplaza los autores de los mensajes por aliases por entrada completa

# Remplaza las menciones de los miembros del grupo en la columna de mensajes en cada entrada en forma de string (por partes)
for (nombre, alias) in zip(nombres, aliases):
    df.Message = df.Message.str.replace(nombre, alias)
    
df.info()
df.head()


error: nothing to repeat at position 0

In [ ]:
df

In [ ]:
import regex
def split_count(text):

    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI for char in word):
            emoji_list.append(word)

    return emoji_list

Autores= df.Author.unique()
total_authors= Autores.shape[0]
total_messages = df.shape[0]
media_messages = df[df['Message'] == '<multimedia omitido>'].shape[0]
df["emoji"] = df["Message"].apply(split_count)
emojis = sum(df['emoji'].str.len())
URLPATTERN = r'(https?://\S+)'
df['urlcount'] = df.Message.apply(lambda x: regex.findall(URLPATTERN, x)).str.len()
links = np.sum(df.urlcount)

In [ ]:
print("Estadísticas del grupo")
print("Autores:", total_authors)
print("Mensajes:", total_messages)
print("Media:",media_messages)
print("Emojis:",emojis)
print("Links",links)

In [ ]:
media_messages_df = df[df['Message'] == '<multimedia omitido>']
messages_df = df.drop(media_messages_df.index)

In [ ]:
messages_df['Letter_Count'] = messages_df['Message'].apply(lambda s : len(s))
messages_df['Word_Count'] = messages_df['Message'].apply(lambda s : len(s.split(' ')))
messages_df["MessageCount"]=1

In [ ]:
messages_df.head()

In [ ]:
l = messages_df.Author.unique()

for i in range(len(l)):
  # Filtering out messages of particular user
  req_df= messages_df[messages_df["Author"] == l[i]]
  # req_df will contain messages of only one particular user
  print(f'Estadísticas de {l[i]} -')
  # shape will print number of rows which indirectly means the number of messages
  print('Mensajes enviados:', req_df.shape[0])
  #Word_Count contains of total words in one message. Sum of all words/ Total Messages will yield words per message
  words_per_message = (np.sum(req_df['Word_Count']))/req_df.shape[0]
  print('Palabras por mensaje:', words_per_message)
  #media conists of media messages
  media = media_messages_df[media_messages_df['Author'] == l[i]].shape[0]
  print('Contenido multimedia enviado:', media)
  # emojis conists of total emojis
  emojis = sum(req_df['emoji'].str.len())
  print('Emojis enviados:', emojis)
  #links consist of total links
  links = sum(req_df["urlcount"])   
  print('Links enviados:', links)   
  print()

In [ ]:
total_emojis_list = list(set([a for b in messages_df.emoji for a in b]))
total_emojis = len(total_emojis_list)
print('Emojis enviados:',total_emojis)

In [ ]:
import collections
total_emojis_list = list([a for b in messages_df.emoji for a in b])
emoji_dict = dict(collections.Counter(total_emojis_list))
emoji_dict = sorted(emoji_dict.items(), key=lambda x: x[1], reverse=True)

emoji_df = pd.DataFrame(emoji_dict, columns=['emoji', 'count'])
emoji_df

In [ ]:
import plotly.express as px
fig = px.pie(emoji_df, values='count', names='emoji',
             title='Distribución de Emojis')
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

In [ ]:
# Crear lista de autores para crear gráficos de uso dee emoji's para cada autor
l = messages_df.Author.unique()
for i in range(len(l)):
  dummy_df = messages_df[messages_df['Author'] == l[i]]
  total_emojis_list = list([a for b in dummy_df.emoji for a in b])
  emoji_dict = dict(collections.Counter(total_emojis_list))
  emoji_dict = sorted(emoji_dict.items(), key=lambda x: x[1], reverse=True)
  print('Emoji Distribution for', l[i])
  author_emoji_df = pd.DataFrame(emoji_dict, columns=['emoji', 'count'])
  fig = px.pie(author_emoji_df, values='count', names='emoji')
  fig.update_traces(textposition='inside', textinfo='percent+label')
  fig.show()

In [ ]:
autor_df = messages_df.groupby("Author").sum()
autor_df

In [ ]:
messages_df['Author'].value_counts().head(15).plot.barh() 
plt.xlabel('Número de mensajes')
plt.ylabel('Autor')

In [ ]:
date_df = messages_df.groupby("Date").sum()
date_df.reset_index(inplace=True)
fig = px.line(date_df, x="Date", y="MessageCount", labels={'Date':'Fecha', 'MessageCount':'Mensajes'}, title='Número de mensajes a través del tiempo.')
fig.update_xaxes(nticks=20)
fig.show()

In [ ]:
def dayofweek(i):
  l = ["Lunes", "Martes", "Miércoes", "Jueves", "Viernes", "Sábado", "Domingo"]
  return l[i];
day_df=pd.DataFrame(messages_df["Message"])
day_df['day_of_date'] = messages_df['Date'].dt.weekday
day_df['day_of_date'] = day_df["day_of_date"].apply(dayofweek)
day_df["messagecount"] = 1
day = day_df.groupby("day_of_date").sum()
day.reset_index(inplace=True)

fig = px.line_polar(day, r='messagecount', theta='day_of_date', line_close=True)
fig.update_traces(fill='toself')
fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
      range=[0,6000]
    )),
  showlegend=False
)
fig.show()

In [ ]:
messages_df['Date'].value_counts().head(10).plot.barh()
plt.xlabel('Número de mensajes')
plt.ylabel('Fecha')

In [ ]:
messages_df['Time'].value_counts().head(15).plot.barh() # Top 10 Times of the day at which the most number of messages were sent
plt.xlabel('Número de mensajes')
plt.ylabel('Hora')

In [ ]:
messages_df.iloc[messages_df['Word_Count'].argmax()]

In [ ]:
text = " ".join(review for review in messages_df.Message)
print ("Hay {} palabras en todos los mensajes.".format(len(text)))

In [ ]:
from nltk.corpus import stopwords
nltk.download("stopwords")
from wordcloud import WordCloud 
stopwords = set(stopwords.words('spanish', 'english')) 
stopwords.update([ "jaja", "com", "http","https", "pues","pue","www", "I'm","bored", "Aburrí", "papol", "fito","pirata"])
wordcloud = WordCloud(stopwords=stopwords, background_color="white").generate(text)

In [ ]:
def plot_cloud(wordcloud):
    # Set figure size
    plt.figure(figsize=(40, 30))
    # Display image
    plt.imshow(wordcloud) 
    # No axis details
    plt.axis("off");

In [ ]:
# Generate word cloud
wordcloud = WordCloud(width = 3000, height = 2000, random_state=1, background_color='black', colormap='Pastel1', collocations=False, stopwords = stopwords).generate(text)
# Plot
plot_cloud(wordcloud)